In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


In [2]:
data_path = '011/text_data/'

ada_joy = data_path + 'ada_joy.csv'
assistance_info = data_path + 'assistance_info.csv'
gaze_positions = data_path + 'gaze_positions.csv'
input_info = data_path + 'input_info.csv'
joint_positions = data_path + 'joint_positions.csv'
myo_emg = data_path + 'myo_emg.csv'
myo_imu = data_path + 'myo_imu.csv'
myo_ori = data_path + 'myo_ori.csv'
pupil_cal_eye0 = data_path + 'pupil_cal_eye0.csv'
pupil_cal_eye1 = data_path + 'pupil_cal_eye1.csv'
pupil_eye0 = data_path + 'pupil_eye0.csv'
pupil_eye1 = data_path + 'pupil_eye1.csv'
robot_position = data_path + 'robot_position.csv'
world_cal_positions = data_path + 'world_cal_positions.csv'

ada_joy_df = pd.read_csv(ada_joy)
assistance_info_df = pd.read_csv(assistance_info)
gaze_positions_df = pd.read_csv(gaze_positions)
input_info_df = pd.read_csv(input_info)
joint_positions_df = pd.read_csv(joint_positions)
myo_emg_df = pd.read_csv(myo_emg)
myo_imu_df = pd.read_csv(myo_imu)
myo_ori_df = pd.read_csv(myo_ori)
pupil_cal_eye0_df = pd.read_csv(pupil_cal_eye0)
pupil_cal_eye1_df = pd.read_csv(pupil_cal_eye1)
pupil_eye0_df = pd.read_csv(pupil_eye0)
pupil_eye1_df = pd.read_csv(pupil_eye1)
robot_position_df = pd.read_csv(robot_position)
world_cal_positions_df = pd.read_csv(world_cal_positions)

In [3]:
from sklearn.preprocessing import StandardScaler

# List of all your dataframes
dfs = [ada_joy_df, assistance_info_df, gaze_positions_df, input_info_df, joint_positions_df, myo_emg_df, myo_imu_df, myo_ori_df, pupil_cal_eye0_df, pupil_cal_eye1_df, pupil_eye0_df, pupil_eye1_df, robot_position_df, world_cal_positions_df]

# Data cleaning: Fill missing values with forward fill method
for df in dfs:
    df.fillna(method='ffill', inplace=True)

# Data normalization: Normalize data to have zero mean and unit variance
scaler = StandardScaler()
for df in dfs:
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

C:\Users\test\AppData\Local\Temp\ipykernel_16824\4193131405.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [4]:
print('joint_positions_df columns: ', joint_positions_df.columns)

joint_positions_df columns:  Index(['timestamp', 'world_index', 'world_index_corrected', 'mico_joint_1_pos',
       'mico_joint_2_pos', 'mico_joint_3_pos', 'mico_joint_4_pos',
       'mico_joint_5_pos', 'mico_joint_6_pos', 'mico_joint_finger_1_pos',
       'mico_joint_finger_2_pos', 'mico_joint_1_vel', 'mico_joint_2_vel',
       'mico_joint_3_vel', 'mico_joint_4_vel', 'mico_joint_5_vel',
       'mico_joint_6_vel', 'mico_joint_finger_1_vel',
       'mico_joint_finger_2_vel', 'mico_joint_1_eff', 'mico_joint_2_eff',
       'mico_joint_3_eff', 'mico_joint_4_eff', 'mico_joint_5_eff',
       'mico_joint_6_eff', 'mico_joint_finger_1_eff',
       'mico_joint_finger_2_eff'],
      dtype='object')


In [5]:
joint_cols = ['mico_joint_1_pos', 'mico_joint_2_pos', 'mico_joint_3_pos', 'mico_joint_4_pos', 'mico_joint_5_pos', 'mico_joint_6_pos', 'mico_joint_finger_1_pos', 'mico_joint_finger_2_pos']

for col in joint_cols:
    joint_positions_df[col + '_acceleration'] = joint_positions_df[col].diff().diff()
    joint_positions_df[col + '_jerk'] = joint_positions_df[col].diff().diff().diff()

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming 'target' is the column you want to predict
X = joint_positions_df.drop('target', axis=1)
y = joint_positions_df['target']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape input to be 3D [samples, timesteps, features]
X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Model building
num_features = len(joint_positions_df.columns)  # number of features in your input data
num_outputs = 1  # replace with the number of outputs you want to predict
model = Sequential()
model.add(LSTM(50, input_shape=(None, num_features)))
model.add(Dense(num_outputs))
model.compile(loss='mean_squared_error', optimizer='adam')